In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py
import glob

from lhcsmapi.Time import Time

In [ ]:
hdf_dir = Path('/eos/project/m/ml-for-alarm-system/private/RB_signals/data')
failed_dir = Path('/eos/project/m/ml-for-alarm-system/private/RB_signals/failed')
context_path = Path('/eos/project/m/ml-for-alarm-system/private/RB_signals/context')

In [ ]:
mp3_fpa_df = pd.read_csv("../data/RB_TC_extract_2021_11_22_processed.csv")

In [ ]:
files = sorted(glob.glob(str(context_path / "*.csv")))
df_context = pd.concat(map(pd.read_csv, files))
df_context["timestamp_fgc_datetime"] = df_context.timestamp_fgc.apply(Time.to_datetime)
df_context.reset_index(drop=True, inplace=True)

In [9]:
files = sorted(glob.glob(str(failed_dir / "*.csv")))
df_failed = pd.concat(map(pd.read_csv, files))
df_failed["timestamp_fgc_datetime"] = df_failed.timestamp_fgc.apply(Time.to_datetime)
df_failed.reset_index(drop=True, inplace=True)

# Missing events I_MEAS 

In [ ]:
df_no_IMEAS = df_context[df_context['PCPM_STATUS.I_MEAS'].isna()][["circuit_name", "timestamp_fgc", "timestamp_fgc_datetime"]]
df_no_IMEAS

In [ ]:
df_failed[df_failed.timestamp_fgc.isin(df_no_IMEAS.timestamp_fgc)]['PCPM'].value_counts()

# Missing events U_DIODE NXCALS

In [ ]:
# find distinct key
df_diode = df_context.filter(regex="DIODE")
print(len(df_diode.columns))

columns = df_diode.columns.values
unique_signals = [c.split("_")[0] for c in columns]
list(dict.fromkeys(unique_signals))

In [6]:
valid_udiode_nxcals_index = df_context.filter(regex="VoltageNXCALS.*U_DIODE").dropna(how='all').index.values
df_context_valid = df_context.iloc[valid_udiode_nxcals_index][["circuit_name", "timestamp_fgc", "timestamp_fgc_datetime"]]
df_context_valid

,circuit_name,timestamp_fgc,timestamp_fgc_datetime
0,RB.A12,1418330714580000000,2014-12-11 21:45:14.580000+01:00
1,RB.A12,1418384764500000000,2014-12-12 12:46:04.500000+01:00
2,RB.A12,1418484670560000000,2014-12-13 16:31:10.560000+01:00
3,RB.A12,1418538814280000000,2014-12-14 07:33:34.280000+01:00
4,RB.A12,1418560853860000000,2014-12-14 13:40:53.860000+01:00
...,...,...,...
766,RB.A81,1424501293480000000,2015-02-21 07:48:13.480000+01:00
767,RB.A81,1428891216980000000,2015-04-13 04:13:36.980000+02:00
768,RB.A81,1428996141520000000,2015-04-14 09:22:21.520000+02:00
769,RB.A81,1436908751420000000,2015-07-14 23:19:11.420000+02:00


In [7]:
df_failed[~df_failed.timestamp_fgc.isin(df_context_valid.timestamp_fgc)].VoltageNXCALS.value_counts()

empty DataFrame returned                        421
Unable to create group (name already exists)     67
Name: VoltageNXCALS, dtype: int64

In [12]:
# get reason of failure for missing files
df_failed_empty = df_failed[(~df_failed.timestamp_fgc.isin(df_context_valid.timestamp_fgc)) & (df_failed.VoltageNXCALS == "empty DataFrame returned")][["VoltageNXCALS", "circuit_name", "timestamp_fgc", "timestamp_fgc_datetime"]]
#df_failed_empty.to_csv("VoltageNXCALS_U_DIODE_empty.csv")
df_failed_empty

,VoltageNXCALS,circuit_name,timestamp_fgc,timestamp_fgc_datetime
30,empty DataFrame returned,RB.A12,1619196160600000000,2021-04-23 18:42:40.600000+02:00
31,empty DataFrame returned,RB.A12,1619244358440000000,2021-04-24 08:05:58.440000+02:00
32,empty DataFrame returned,RB.A12,1619349125540000000,2021-04-25 13:12:05.540000+02:00
33,empty DataFrame returned,RB.A12,1619374541080000000,2021-04-25 20:15:41.080000+02:00
34,empty DataFrame returned,RB.A12,1619417171040000000,2021-04-26 08:06:11.040000+02:00
...,...,...,...,...
822,empty DataFrame returned,RB.A81,1629782243260000000,2021-08-24 07:17:23.260000+02:00
823,empty DataFrame returned,RB.A81,1629831279760000000,2021-08-24 20:54:39.760000+02:00
824,empty DataFrame returned,RB.A81,1629911725660000000,2021-08-25 19:15:25.660000+02:00
825,empty DataFrame returned,RB.A81,1629956195440000000,2021-08-26 07:36:35.440000+02:00


In [16]:
1619196160600000000 ==1619196160600000000

True

## Open h5 files

In [15]:
file_dir = Path('/eos/project/m/ml-for-alarm-system/private/RB_signals/data')

for index, row in df_context.iterrows():   
    circuit_type = 'RB' #row['circuit_type']
    circuit_name = 'RB.A12' #row['circuit_name']
    timestamp_fgc = 1619196160600000000 #row['timestamp_fgc']
    
    filename = f"{circuit_type}_{circuit_name}_{timestamp_fgc}.hdf5"
    
    with h5py.File(file_dir / filename, "r") as f:
        
        k = np.array(f.keys())
        print(k)

        #plt.plot(np.array(f["VoltageNXCALS"]["MB.A31L2:U_DIODE_RB"].get("values")))
        
    break

<KeysViewHDF5 ['CurrentVoltageDiodeLeadsPM', 'EETResPM', 'EEUDumpResPM', 'Leads', 'PCPM', 'QHPM', 'VoltageLogicIQPS', 'VoltageLogicNQPS']>


# Missing events U_DIODE PM

In [ ]:
df_context.filter(regex="VoltageLogicNQPS.*U_DIODE").dropna(how='all')

In [ ]:
valid_udiode_pm_index = df_context.filter(regex="VoltageLogicNQPS.*U_DIODE").dropna(how='all').index.values
df_context_valid = df_context.iloc[valid_udiode_pm_index][["circuit_name", "timestamp_fgc", "timestamp_fgc_datetime"]]
df_context_valid

In [ ]:
df_failed[~df_failed.timestamp_fgc.isin(df_context_valid.timestamp_fgc)].VoltageLogicNQPS.value_counts()